In [8]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import datetime



In [9]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)




In [10]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [11]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7598, 8)

In [12]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7294, 5)

In [13]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7604, 10)

In [15]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

fin empresas y comienzo de rescate de pacientes


In [16]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1808793, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [17]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1417165, 4)

In [18]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1417165, 13)

In [19]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [20]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [21]:
#pacientes_empresas22.shape

In [22]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [23]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [24]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [25]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BCI Canal Individual"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [26]:
Pacientes_Archivo.head(10)

,poliza,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,sexo,PARENTESCO,NOMBRE_ASEGURADO,nombres,apellidos,nacimiento,email,direccion,celular
0,2597777,100,7,100,7,100-7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,JOSE,R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,VASCO DE GAMA NRO 4691,NaN
1,2601791,15583606,7,15583606,7,15583606-7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,JESUS,NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465.0
2,2601791,15583606,7,15797990,6,15797990-6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,Gajardo,Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488.0
3,2601791,15583606,7,20788055,8,20788055-8,FEMENINO,HIJA,YAKSA ELIENETT REY GAJARDO,YAKSA,ELIENETT REY GAJARDO,2001-07-13,NaN,SANTIAGO,NaN
4,2601791,15583606,7,22274306,0,22274306-0,FEMENINO,HIJA,ALONDRA ESTEFANIA REY GAJARDO,ALONDRA,ESTEFANIA REY GAJARDO,2006-11-30,NaN,SANTIAGO,NaN
5,2601791,15583606,7,26105686,0,26105686-0,MASCULINO,HIJO,FACUNDO EMILIANO REY GAJARDO,FACUNDO,EMILIANO REY GAJARDO,2018-02-03,NaN,SANTIAGO,NaN
6,2602256,19636480,3,19636480,3,19636480-3,MASCULINO,TITULAR MASCULINO,FELIPE IGNACIO BISAMA DUIJVESTEIJN,FELIPE,IGNACIO BISAMA DUIJVESTEIJN,1997-06-06,p.bisama@gmail.com,SANTIAGO SN,963361410.0
7,2602285,15451267,5,15451267,5,15451267-5,FEMENINO,TITULAR MASCULINO,MARISOL DEL PILAR ACEVEDO ALARCON,MARISOL,DEL PILAR ACEVEDO ALARCON,1982-02-07,NaN,CALLE TAIHUEL 07037 PORTEZUELA OTE,987447006.0
8,2602285,15451267,5,14067401,K,14067401-K,MASCULINO,ESPOSO,RODRIGO ALEXIS YAÑEZ SANHUEZA,RODRIGO,ALEXIS YAÑEZ SANHUEZA,1981-01-09,rodrigo28y@gmail.com,SN,988449453.0
9,2602285,15451267,5,24318085,6,24318085-6,FEMENINO,HIJO,VICENTE LEONARDO YAÑEZ ACEVEDO,VICENTE,LEONARDO YAÑEZ ACEVEDO,2013-06-28,NaN,SN,NaN


In [27]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,sexo,PARENTESCO,NOMBRE_ASEGURADO,nombres,apellidos,nacimiento,email,direccion,celular,rut
0,2597777,100,7,100,7,100-7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,JOSE,R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,VASCO DE GAMA NRO 4691,NaN,1007
1,2601791,15583606,7,15583606,7,15583606-7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,JESUS,NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465.0,155836067
2,2601791,15583606,7,15797990,6,15797990-6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,Gajardo,Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488.0,157979906
3,2601791,15583606,7,20788055,8,20788055-8,FEMENINO,HIJA,YAKSA ELIENETT REY GAJARDO,YAKSA,ELIENETT REY GAJARDO,2001-07-13,NaN,SANTIAGO,NaN,207880558
4,2601791,15583606,7,22274306,0,22274306-0,FEMENINO,HIJA,ALONDRA ESTEFANIA REY GAJARDO,ALONDRA,ESTEFANIA REY GAJARDO,2006-11-30,NaN,SANTIAGO,NaN,222743060


In [29]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21724\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,sexo,PARENTESCO,NOMBRE_ASEGURADO,nombres,apellidos,nacimiento,email,direccion,celular,rut
0,2597777,100,7,100,7,100-7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,JOSE,R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,VASCO DE GAMA NRO 4691,NaN,1007
1,2601791,15583606,7,15583606,7,15583606-7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,JESUS,NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465.0,155836067
2,2601791,15583606,7,15797990,6,15797990-6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,Gajardo,Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488.0,157979906
3,2601791,15583606,7,20788055,8,20788055-8,FEMENINO,HIJA,YAKSA ELIENETT REY GAJARDO,YAKSA,ELIENETT REY GAJARDO,2001-07-13,NaN,SANTIAGO,NaN,207880558
4,2601791,15583606,7,22274306,0,22274306-0,FEMENINO,HIJA,ALONDRA ESTEFANIA REY GAJARDO,ALONDRA,ESTEFANIA REY GAJARDO,2006-11-30,NaN,SANTIAGO,NaN,222743060


In [30]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21724\192164855.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21724\192164855.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [31]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(1176, 26)

In [32]:
Pacientes_con_id.head()

,poliza,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,sexo,PARENTESCO,NOMBRE_ASEGURADO,nombres,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,2597777,100,7,100,7,100-7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,Jose,...,True,0018c00002nkxRUAAY,1007,Jose R Friaz Ruiz,RUT,None,None,None,None,92893045
1,2601791,15583606,7,15583606,7,15583606-7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,Jesus,...,True,0018c00002mVtq4AAC,155836067,Jesus Noe Rey Acevedo,RUT,None,None,None,None,Nan
2,2601791,15583606,7,15797990,6,15797990-6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,Gajardo,...,True,0018c00002mVtswAAC,157979906,Cynthia Del Pilar Gajardo Pradenas,RUT,None,None,None,None,Nan
3,2601791,15583606,7,20788055,8,20788055-8,FEMENINO,HIJA,YAKSA ELIENETT REY GAJARDO,Yaksa,...,True,0018c00002mVtq1AAC,207880558,Yaksa Elienett Rey Gajardo,RUT,None,None,None,None,Nan
4,2601791,15583606,7,22274306,0,22274306-0,FEMENINO,HIJA,ALONDRA ESTEFANIA REY GAJARDO,Alondra,...,True,0018c00002mVtq2AAC,222743060,Alondra Estefania Rey Gajardo,RUT,None,None,None,None,Nan


In [33]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21724\628618797.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
1077,False,Matas,Fabin Ahumada Cereño,22163151K,yasna1984cereno@gmail.com,2006-07-14,Hombre,No Informado,,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1078,False,Regis,Emilio Lizama Caceres,121568306,chequenino@gmil.com,1970-05-03,Hombre,No Informado,,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1079,False,Andres,Benoit Gaete,170867211,abenoitg@gmail.com,1989-03-31,Hombre,Santiago,992237.,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1080,False,Barbara,Ellis Escudero,183959441,no informado,1993-06-25,Mujer,No Informado,,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1081,False,Soto,Lopez Marina Soledad,134005475,soledad.sotolopez@gmail.com,1978-09-07,Mujer,Santiago,942383291.,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [34]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
#clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [35]:
""" print(dia_hoy) """

' print(dia_hoy) '

In [36]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
735473,a1Y8c00000CzGHSEA3,0018c00002mVtphAAC,a1W8c000008TCWxEAO,BCI Canal Individual,0018c00002mVq1IAAS,Canal Individual,None,BCI Canal Individual,Canal Individual,None,None,EMP10900,A-10463
735474,a1Y8c00000CzGHTEA3,0018c00002mVtpiAAC,a1W8c000008TCWxEAO,BCI Canal Individual,0018c00002mVq1IAAS,Canal Individual,None,BCI Canal Individual,Canal Individual,None,None,EMP10900,A-10463
735475,a1Y8c00000CzGHUEA3,0018c00002mVtpjAAC,a1W8c000008TCWxEAO,BCI Canal Individual,0018c00002mVq1IAAS,Canal Individual,None,BCI Canal Individual,Canal Individual,None,None,EMP10900,A-10463
735476,a1Y8c00000CzGHVEA3,0018c00002mVtpkAAC,a1W8c000008TCWxEAO,BCI Canal Individual,0018c00002mVq1IAAS,Canal Individual,None,BCI Canal Individual,Canal Individual,None,None,EMP10900,A-10463
735477,a1Y8c00000CzGHWEA3,0018c00002mVtplAAC,a1W8c000008TCWxEAO,BCI Canal Individual,0018c00002mVq1IAAS,Canal Individual,None,BCI Canal Individual,Canal Individual,None,None,EMP10900,A-10463


In [37]:
campaña_seleccionada.shape

(1177, 13)

In [38]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [39]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [40]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)


In [41]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)